In [1]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# # 读入训练和测试数据
# ## 本地，同时平衡正反例
# def balance(train_pro):
#     ## 从正例中随机取出比例为a的数据，划分
#     data = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')
#     data['date'] = data['clickTime'].apply(lambda x: int(x/10000))
#     data = data[data['date']<30] #去掉30号那天的数据
#     train_size_p = int(data[data['label']==1]['label'].count()*train_pro)
#     train_size_n = int(train_size_p * 38.2751) #原始样本中负例是正例的这个倍数(所有样本)
#     sampler1 = np.random.permutation(data[data['label']==1]['label'].count())
#     index1 = data[data['label']==1].index[sampler1] #重排序后的索引
#     train1 = data[data['label']==1].ix[index1[:train_size_p]]
#     test1 = data[data['label']==1].ix[index1[train_size_p:]]
#     ## 从负例中随机取出相同数量的数据，划分
#     sampler0 = np.random.permutation(data[data['label']==0]['label'].count())
#     index0 = data[data['label']==0].index[sampler0] #重排序后的索引
#     train0 = data[data['label']==0].ix[index0[:train_size_n]]
#     test0 = data[data['label']==0].ix[index0[train_size_n:]]
#     ## 构建训练集和测试集
#     data_train = pd.concat([train1, train0])
#     data_test = pd.concat([test1, test0])
#     return data_train, data_test
# # 获取正反例
# trainData, testData = balance(0.8)

In [2]:
# 预测
trainData = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')
testData = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\test.csv')

In [3]:
testData.head()

,instanceID,label,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,1,-1,310000,3745,1164848,3451,1,3
1,2,-1,310000,2284,2127247,1613,1,3
2,3,-1,310000,1456,2769125,5510,2,1
3,4,-1,310000,4565,9762,4113,2,3
4,5,-1,310000,49,2513636,3615,1,3


In [4]:
# 读入基础数据
for i,dt in enumerate([trainData,testData]):
    dt['date'] = dt['clickTime'].apply(lambda x: int(x/10000))
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\user.csv'), on='userID')
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv'), on='creativeID')
    dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\position.csv'), on='positionID')
    if i == 0:
        trainData = dt
    elif i == 1:
        testData = dt

In [5]:
testData = trainData[trainData['date']==28]
trainData = trainData[trainData['date']<28]

In [ ]:
# # 对点击时间进行处理
# def dealTime(clicktime):
#     hour = int(str(clicktime)[2:4])
#     minute = int(str(clicktime)[4:6])
#     x = int(60*hour+minute)/30
#     if x in [18,19,20]:
#         return 1
#     elif x in [21,22,23]:
#         return 2
#     elif x in [4,24]:
#         return 3
#     elif x in [25,26]:
#         return 4
#     elif x in [2,3,6,10,27,31,47]:
#         return 5
#     elif x in [0,1,5,7,8,9,11,28,30,44,45]:
#         return 6
#     elif x in [13,14]:
#         return 8
#     elif x in [15]:
#         return 9
#     else: #中下的最多，所以else来处理
#         return 7

In [ ]:
testData.head()

In [ ]:
# # 读入更多特征
# for i,dt in enumerate([trainData,testData]):
#     ## 读入age
#     dt.drop(['age'], axis=1, inplace=True) #有些原有字段中有的但经过处理的，先删除，再读入
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\age.csv'), on='userID')
#     ## residence提取省份
# #     dt['residence'] = dt['residence'].apply(lambda x: int(str(x)[-4:-2]) if x>0 else 0)
#     ## 读入appCategory
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\appCategory.csv'), on='creativeID')
#     ## 读入clickTime_hour
#     dt['clickTime_hour'] = dt['clickTime'].apply(dealTime)
#     ## 读入if_appinstall
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\if_appinstall.csv'), on='userID')
#     ## 读入if_appaction
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\if_appaction.csv'), on='userID')
#     ## 读入count_appinstall
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\count_appinstall.csv'), on='userID')
#     ## 读入count_appaction
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\count_appaction.csv'), on='userID')
#     ## 读入count_user_ad
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\count_user_ad.csv'), on='userID')
#     ## 读入vote
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\user_single_vote_simple1008_r2.csv'), on=['userID','appID'], how='left')
#     ## 读入lastNCVR_advertiserID
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_advertiserID.csv'), on=['date', 'creativeID'])
#     ## 读入lastNCVR_appID
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_appID.csv'), on=['date', 'creativeID'])
#     ## 读入lastNCVR_camgaignID
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_camgaignID.csv'), on=['date', 'creativeID'])
#     ## 读入lastNCVR_positionType
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_positionType.csv'), on=['date', 'positionID'])
#     ## 读入lastNCVR_sitesetID
#     dt = pd.merge(dt, pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\lastNCVR_sitesetID.csv'), on=['date', 'positionID'])
#     dt = dt.fillna(0)
#     if i == 0:
#         trainData = dt
#     elif i == 1:
#         testData = dt

In [ ]:
# # 连续特征标准化
# import sklearn.preprocessing as preprocessing
# # features = ['lastNCVR_appID', 'lastNCVR_advertiserID','lastNCVR_positionType', 'lastNCVR_sitesetID','lastNCVR_camgaignID',
# #            'count_appinstall', 'count_appaction','userAdCount']
# features = ['lastNCVR_appID', 'lastNCVR_advertiserID','lastNCVR_positionType', 'lastNCVR_sitesetID','lastNCVR_camgaignID',
#            'count_appinstall', 'count_appaction','voteMinus']
# for i,feature in enumerate(features):
#     # 训练集中，保存scale
# #     scaler = preprocessing.StandardScaler()
# #     scaler.fit(trainData[feature].values.reshape(-1,1))
# #     joblib.dump(scaler, 'C:\\Users\\work\\Desktop\\tencent\\clf\\scaler' + feature + '.pkl', compress=3)
#     # 测试集中，读取scale
#     scaler = joblib.load('C:\\Users\\work\\Desktop\\tencent\\clf\\scaler' + feature + '.pkl')
#     # 线上测试时，将最后一天的cvr提高
#     s = 1
# #     if feature == 'lastNCVR_appID': #1天
# #         s = 1.34971374
# #     elif feature == 'lastNCVR_advertiserID': #1天
# #         s = 1.34971374
# #     elif feature == 'lastNCVR_camgaignID': #1天
# #         s = 1.34971374
# #     elif feature == 'lastNCVR_sitesetID': #2天
# #         s = 1.173481173
# #     elif feature == 'lastNCVR_positionType': #4天
# #         s = 1.096835297
#     x_train = (trainData[feature]*s).values.reshape(-1,1)
#     x_test = (testData[feature]*s).values.reshape(-1,1)
# #     x_train = scaler.transform((trainData[feature]*s).values.reshape(-1,1))
# #     x_test = scaler.transform((testData[feature]*s).values.reshape(-1,1))
#     if i == 0:
#         X_train, X_test = x_train, x_test
#     else:
#         X_train, X_test = np.hstack((X_train, x_train)), np.hstack((X_test, x_test))

In [ ]:
# # 不onehot的离散变量
# # features = ['creativeID', 'adID', 'camgaignID', 'advertiserID']
# features = ['connectionType', 'telecomsOperator', 'marriageStatus','haveBaby', 'appPlatform', 
#             'education', 'sitesetID', 'appCategory', 'clickTime_hour','appID','age','positionType',
#             'advertiserID', 'adID','if_appinstall','if_appaction']
# for feature in features:
#     x_train = trainData[feature].values.reshape(-1,1)
#     x_test = testData[feature].values.reshape(-1,1)
#     X_train, X_test = np.hstack((X_train, x_train)), np.hstack((X_test, x_test))

In [6]:
# 离散变量onehot化
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
# features = ['connectionType', 'telecomsOperator', 'marriageStatus','haveBaby', 'appPlatform', 
#             'education', 'sitesetID', 'appCategory', 'clickTime_hour','appID','age','gender','positionType',
#             'if_appinstall','if_appaction']
#             'creativeID', 'adID', 'camgaignID', 'advertiserID', 'positionID']
features = ["creativeID", "adID", "camgaignID", "advertiserID", "appID", "appPlatform"]
for i,feature in enumerate(features):
    # 训练集中，保存enc
    enc = OneHotEncoder()
    enc.fit(trainData[feature].values.reshape(-1,1))
    joblib.dump(enc, 'C:\\Users\\work\\Desktop\\tencent\\clf\\enc' + feature + '.pkl', compress=3)
    # 测试集中，读取enc
#     enc = joblib.load('C:\\Users\\work\\Desktop\\tencent\\clf\\enc' + feature + '.pkl')
    x_train = enc.transform(trainData[feature].values.reshape(-1, 1))
    x_test = enc.transform(testData[feature].values.reshape(-1, 1))
    if i == 0:
        X_train, X_test = x_train, x_test
    else:
        X_train, X_test = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test))

In [7]:
# y值
y_train = trainData['label'].values
y_test = testData['label'].values

# 训练GBDT树

## 1.本地训练与测试

In [8]:
# 划分训练集和测试集
from sklearn import cross_validation
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingClassifier

# # # 将训练集切分为两部分，一部分用于训练GBDT模型，另一部分输入到训练好的GBDT模型生成GBDT特征，然后作为LR的特征。这样分成两部分是为了防止过拟合。
# X_train_gbdt, X_train_lr, y_train_gbdt, y_train_lr = cross_validation.train_test_split(X_train, y_train, test_size=0.5)
# # # 调用GBDT分类模型。
# grd = GradientBoostingClassifier(n_estimators=30,learning_rate=0.1, max_depth=8,max_features='sqrt',subsample=0.8)
# # # 使用X_train训练GBDT模型，后面用此模型构造特征
# grd.fit(X_train_gbdt, y_train_gbdt)

In [9]:
# 调用LR分类模型
grd_lm = linear_model.LogisticRegressionCV(Cs=10, penalty='l2', tol=1e-6, n_jobs=-1, cv=5)
# grd_enc = OneHotEncoder() #one-hot编码 
# grd_enc.fit(grd.apply(X_train)[:, :, 0]) #每个数据，在每棵树，叶节点index
# 使用训练好的GBDT模型构建特征，然后将特征经过one-hot编码作为新的特征输入到LR模型训练。
# grd_lm.fit(grd_enc.transform(grd.apply(X_train)[:, :, 0]), y_train_lr)
grd_lm.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=1e-06, verbose=0)

In [10]:
# 评估模型结果
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll
from sklearn import metrics
# predict = grd_lm.predict(grd_enc.transform(grd.apply(X_test)[:, :, 0]))
predict = grd_lm.predict(X_test)
print 'P&R'
print metrics.classification_report(y_test, predict)
print 'loss:'
# predict_pro = grd_lm.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0]))[:, 1]
predict_pro = grd_lm.predict_proba(X_test)[:,1]
print logloss(y_test, predict_pro)

P&R
             precision    recall  f1-score   support

          0       0.97      1.00      0.99    271291
          1       0.00      0.00      0.00      7343

avg / total       0.95      0.97      0.96    278634

loss:
0.116579080531


C:\Users\work\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
X_test.shape

In [ ]:
# 保存分类器
# joblib.dump(grd_enc, 'C:\\Users\\work\\Desktop\\tencent\\clf\\gbdt_onehot_baseline.pkl', compress=3)
# joblib.dump(grd, 'C:\\Users\\work\\Desktop\\tencent\\clf\\gbdt_5.pkl', compress=3)
joblib.dump(grd_lm, 'C:\\Users\\work\\Desktop\\tencent\\clf\\gbdt_lr_baseline.pkl', compress=3)

In [ ]:
from sklearn.metrics import roc_curve
# # 用训练好的LR模型多X_test做预测
# y_pred_grd_lm = grd_lm.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0]))[:, 1]
# # 根据预测结果输出
# fpr_grd_lm, tpr_grd_lm, _ = roc_curve(y_test, y_pred_grd_lm)
import matplotlib.pyplot as plt
plt.plot(range(len(fpr_grd_lm)), fpr_grd_lm)
plt.plot(range(len(tpr_grd_lm)), tpr_grd_lm)
plt.show()

In [ ]:
print y_test[:10]
print predict_pro[:10]

In [ ]:
predict_pro.sum()/len(predict_pro)

In [ ]:
# cf = pd.DataFrame({'columns':data.columns[1:], 'importance':list(grd.feature_importances_.T)})
cf = pd.DataFrame({'importance':list(grd.feature_importances_.T)})
cf.sort_values(by=['importance'],ascending=False, inplace=True)
print cf

In [ ]:
cf.to_csv('C:\\Users\\work\\Desktop\\tencent\\clf\\coef_6.csv',index=False)

In [ ]:
cf

# 预测

In [ ]:
# 读取分类器
# grd_enc = joblib.load('C:\\Users\\work\\Desktop\\tencent\\clf\\gbdt_onehot_baseline.pkl')
# grd = joblib.load('C:\\Users\\work\\Desktop\\tencent\\clf\\gbdt_5.pkl')
clf = joblib.load('C:\\Users\\work\\Desktop\\tencent\\clf\\gbdt_lr_baseline.pkl')

In [ ]:
print X_train.shape

In [ ]:
# 用训练数据大概评估下模型结果
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll
from sklearn import metrics
# predict = clf.predict(grd_enc.transform(grd.apply(X_train)[:, :, 0]))
predict = clf.predict(X_train)
print 'P&R'
print metrics.classification_report(y_train, predict)
print 'loss:'
# predict_pro = clf.predict_proba(grd_enc.transform(grd.apply(X_train)[:, :, 0]))[:, 1]
predict_pro = clf.predict_proba(X_train)[:,1]
print logloss(y_train, predict_pro)

In [ ]:
# 输出预测结果
predict_pro = clf.predict_proba(X_test)[:,1]
# predict_pro = clf.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0]))[:, 1]

In [ ]:
print clf.predict_proba(grd_enc.transform(grd.apply(X_test[:5])[:, :, 0]))

In [ ]:
predict_pro.sum()/len(predict_pro)

In [ ]:
output = pd.DataFrame({'instanceID': testData['instanceID'].values,'prob':predict_pro})
output = output.sort_values(['instanceID'])

In [ ]:
output.head()

In [ ]:
output.to_csv('C:\\Users\\work\\Desktop\\tencent\\predict\\gbdt_lr5.csv', index=False)

In [ ]:
d = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\predict\\gbdt_lr5.csv')

In [ ]:
d.head(100)

In [ ]:
d['prob'].mean()